In [1]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
model=load_model("model.h5")

loading back the onehot encoded and label encoded

In [2]:
with open("le_gender.pkl","rb") as file:
    le_gender=pickle.load(file)
with open("ohe.pkl" ,"rb") as file:
    ohe=pickle.load(file)
with open("scaler.pkl" ,"rb") as file:
    scaler=pickle.load(file)

In [3]:
input_data={
    "CreditScore": 600,
    "Geography" :"France",
    "Gender" :"Male",
    "Age":40,
    "Tenure":3,
    "Balance": 60000,
    "NumofProducts":2,
    "HasCrCard":"1",
    "IsActiveMember":1,
    "Estimated Salary":50000
    
}

In [4]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,Estimated Salary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
geo_encoded=ohe.transform([[input_data["Geography"]]]).toarray()
geo_encoded

d:\MyLearning\annclassification\env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [6]:
ge_encoded_df=pd.DataFrame(geo_encoded,columns=ohe.get_feature_names_out(["Geography"]))

ge_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df["Gender"]=le_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,Estimated Salary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
input_df=pd.concat([input_df.drop('Geography',axis=1),ge_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,Estimated Salary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
expected_columns = scaler.feature_names_in_
print("expected:",expected_columns,"input_df:",input_df)
input_df = input_df.reindex(columns=expected_columns)


expected: ['CreditScore' 'Gender' 'Age' 'Tenure' 'Balance' 'NumOfProducts'
 'HasCrCard' 'IsActiveMember' 'EstimatedSalary' 'Exited'
 'Geography_France' 'Geography_Germany'] input_df:    CreditScore  Gender  Age  Tenure  Balance  NumofProducts HasCrCard  \
0          600       1   40       3    60000              2         1   

   IsActiveMember  Estimated Salary  Geography_France  Geography_Germany  \
0               1             50000               1.0                0.0   

   Geography_Spain  
0              0.0  


In [10]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
                nan,  0.64920267,  0.97481699,         nan,         nan,
         1.00150113, -0.57946723]])

making predictions

In [12]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 3s 3s/step


array([[0.37025064]], dtype=float32)

To get the prediction probability

In [13]:
prediction_proba=prediction[0][0]
if prediction_proba <0.5:
    print("customer is likely to churn")
else:
    print("customer is not likely to churn")

customer is likely to churn
